<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/ann_tensorflow(classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder
import pickle

In [94]:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# data transformation


In [96]:
#drop unwanted column
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [97]:
print(df['Gender'].unique())
gender_encoder=LabelEncoder()
df['Gender']=gender_encoder.fit_transform(df['Gender'])
df.head()

['Female' 'Male']


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [98]:
print(df['Geography'].unique())
print('len: ',len(df['Geography'].unique()))
geo_encoder=OneHotEncoder()
temp_df=geo_encoder.fit_transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp_df,columns=geo_encoder.get_feature_names_out())
temp_df.head()

['France' 'Spain' 'Germany']
len:  3


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [99]:
x=pd.concat([temp_df,df.drop(columns=['Geography','Exited'])],axis=1)
y=df['Exited']
x.shape

(10000, 12)

# checking for imbalance dataset


In [100]:
y.value_counts()

,count
Exited,
0,7963
1,2037


#  scaling all the feature using minmaxscaler

In [101]:
scaler=MinMaxScaler()
x_transformed=scaler.fit_transform(x)
x_transformed[:2,:]

array([[1.        , 0.        , 0.        , 0.538     , 0.        ,
        0.32432432, 0.2       , 0.        , 0.        , 1.        ,
        1.        , 0.50673489],
       [0.        , 0.        , 1.        , 0.516     , 0.        ,
        0.31081081, 0.1       , 0.33403148, 0.        , 0.        ,
        1.        , 0.56270874]])

#train test spliting

In [102]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=.2,random_state=42)
np.unique(y_train, return_counts=True)

(array([0, 1]), array([6356, 1644]))

In [103]:
# from tensorflow.keras import Model
# from tensorflow.keras.layers import Dense,Input
# input=Input(shape=(12,))
# hidden=Dense(64,activation=tf.keras.activations.leaky_relu)(input)
# hidden=Dense(32,activation=tf.keras.activations.leaky_relu)(hidden)
# hidden=Dense(16,activation=tf.keras.activations.leaky_relu)(hidden)
# hidden=Dense(5,activation=tf.keras.activations.leaky_relu)(hidden)
# output=Dense(1)(hidden)
# model=Model(inputs=input,outputs=output)
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=.001,),metrics=[tf.keras.metrics.Precision,tf.keras.metrics.BinaryAccuracy])
# model.summary()

In [104]:
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# print(type(y_train))
# class_weight=compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train.values)
# class_weight_dict = {i: class_weight[i] for i in range(len(class_weight))}
# earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
# model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=1024,epochs=100,callbacks=[earlystopping],class_weight=class_weight_dict)

# handling imbalanced dataset(oversampling)

In [105]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [106]:
y_train_resampled.value_counts()

,count
Exited,
0,6356
1,6356


# designing model and compiling

In [124]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Input
input=Input(shape=(12,))
hidden=Dense(64,activation=tf.keras.activations.relu)(input)
hidden=Dense(32,activation=tf.keras.activations.relu)(hidden)
hidden=Dense(16,activation=tf.keras.activations.relu)(hidden)
hidden=Dense(5,activation=tf.keras.activations.relu)(hidden)
output=Dense(1)(hidden)
model=Model(inputs=input,outputs=output)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=.01,beta_1=.8,beta_2=.9),metrics=[tf.keras.metrics.Precision,tf.keras.metrics.BinaryAccuracy])


# training model

In [125]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=80,restore_best_weights=True)
model.fit(x_train_resampled,y_train_resampled,validation_data=(x_test,y_test),batch_size=1024,epochs=100,callbacks=[earlystopping])

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - binary_accuracy: 0.5532 - loss: 0.6583 - precision_20: 0.6690 - val_binary_accuracy: 0.7670 - val_loss: 0.5772 - val_precision_20: 0.4129
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - binary_accuracy: 0.6744 - loss: 0.5885 - precision_20: 0.7546 - val_binary_accuracy: 0.7575 - val_loss: 0.6197 - val_precision_20: 0.4241
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - binary_accuracy: 0.7133 - loss: 0.5491 - precision_20: 0.7705 - val_binary_accuracy: 0.7875 - val_loss: 0.5573 - val_precision_20: 0.4726
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - binary_accuracy: 0.7482 - loss: 0.4972 - precision_20: 0.8112 - val_binary_accuracy: 0.8275 - val_loss: 0.4733 - val_precision_20: 0.5541
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - binary_accuracy: 0.7628 - loss: 0.4637 - precision_20: 0.8194 - val_binary_accuracy: 0.8385 - val_loss: 0.4341 - val_precision_20: 0.5907
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0